Cross Validation 1

size =  72684

test size 4k+

epoch-5

apply sigmoid (prediction probabilities are logits)

groups-9

target_list = ['Camera', 'Location', 'Microphone', 'Contacts', 'Storage', 'Phone', 'SMS', 'Call_Log', 'Calendar']

threshold-tuning = yes

df_2k = df[(df['Rating'] >= 4.0) & (df['Maximum_Installs'] >= 20000)]

accuracy score : F1 score (micro) and ROC_AUC score

## 1. Setup

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
print(tf.__version__)

2.8.0


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


## 2. Load Dataset

In [ ]:
## import libraries

import itertools
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn import preprocessing
%matplotlib inline

In [ ]:
## uploading csv files on drive (to avoid uploading on colab in every session)

from google.colab import drive
drive.mount("/content/drive/")

## drive path
train_path = "/content/drive/MyDrive/MetadataCSV/data_72684/CV_df_train_1.csv"
val_path = "/content/drive/MyDrive/MetadataCSV/data_72684/CV_df_val_1.csv"
test_path = "/content/drive/MyDrive/MetadataCSV/test_dataset.csv"

Mounted at /content/drive/


In [ ]:
df_train = pd.read_csv(train_path) 
df_val = pd.read_csv(val_path)
df_test = pd.read_csv(test_path)

print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(58148, 23)
(14536, 23)
(4624, 23)


In [ ]:
df_train.head(2)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Sensors,Camera,...,Contacts,SMS,Storage,Phone,Get_Accounts,Call_Log,desc_length,Clean_Description,clean_desc_length,token_length
0,Face Chat,com.facechat.live,Social,4.1,7306795,False,Enjoy chatting? Social and Video Chat on Face ...,https://sites.google.com/view/xender-chat-term...,0,1,...,0,0,1,1,0,0,1090,enjoy chatting? social and video chat on face ...,1025,197
1,Running - Calorie Counter,com.sdgcode.runningcaloriecounter,Health & Fitness,4.1,286133,False,"Running Counter - Calorie Counter app, your wa...",http://app.sdgcode.com/privacy-policy/,0,0,...,0,0,0,0,0,0,845,"running counter calorie counter app, your way ...",816,162


In [ ]:
target_list = ['Camera', 'Location', 'Microphone', 'Contacts', 'Storage', 'Phone', 'SMS', 'Call_Log', 'Calendar']

In [ ]:
# getting number of nonzeros in each column
df_train[target_list].astype(bool).sum(axis=0)

Camera         9200
Location       9109
Microphone     5402
Contacts       6330
Storage       28500
Phone          8924
SMS             279
Call_Log        169
Calendar       1031
dtype: int64

In [ ]:
df_val[target_list].astype(bool).sum(axis=0)

Camera        2337
Location      2413
Microphone    1384
Contacts      1604
Storage       7108
Phone         2312
SMS             73
Call_Log        58
Calendar       264
dtype: int64

In [ ]:
df_test[target_list].astype(bool).sum(axis=0)

Camera         745
Location       716
Microphone     436
Contacts       500
Storage       2402
Phone          652
SMS             11
Call_Log         6
Calendar        90
dtype: int64

## 3. Data Preprocess

#### 3.1 Clean Text

In [ ]:
import nltk

nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

In [ ]:
df_train["Clean_Description"] = df_train["Clean_Description"].map(remove_stopwords)
df_val["Clean_Description"] = df_val["Clean_Description"].map(remove_stopwords)
df_test["Clean_Description"] = df_test["Clean_Description"].map(remove_stopwords)

In [ ]:
import string

def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [ ]:
df_train["Clean_Description"] = df_train["Clean_Description"].map(lambda x: remove_punct(x))
df_val["Clean_Description"] = df_val["Clean_Description"].map(lambda x: remove_punct(x))
df_test["Clean_Description"] = df_test["Clean_Description"].map(lambda x: remove_punct(x))

#### 3.2 Create Corpus

In [ ]:
from keras.layers import *

In [ ]:
import os
import sys
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Conv2D, Input, Embedding, Reshape, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Conv1D
from keras.layers import MaxPool1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [ ]:
from nltk.tokenize import word_tokenize
import nltk


def create_corpus_tk(df):
    corpus = []
    for text in df["Clean_Description"]:
        words = [word.lower() for word in word_tokenize(text)]
        corpus.append(words)
    return corpus

In [ ]:
df_T = df_train.append(df_val)
df_T.reset_index(drop=True,inplace=True)

In [ ]:
corpus = create_corpus_tk(df_T)

In [ ]:
num_words = len(corpus)
print(num_words)

72684


In [ ]:
corpus[0]

['enjoy',
 'chatting',
 'social',
 'video',
 'chat',
 'face',
 'chat',
 'face',
 'chat',
 'opens',
 'new',
 'doors',
 'video',
 'chatting',
 'people',
 'start',
 'chatting',
 'text',
 'audio',
 'video',
 'options',
 'anytime',
 'anywhere',
 'novel',
 'fun',
 'features',
 'waiting',
 'discover',
 'enjoy',
 'time',
 'face',
 'chat',
 'now',
 'face',
 'chat',
 'chat',
 'people',
 'social',
 'friends',
 'real',
 'time',
 'translation',
 'private',
 'secure',
 'video',
 'chat',
 'quick',
 'call',
 'easy',
 'login',
 'instant',
 'messages',
 'im',
 'beauty',
 'effect',
 'vip',
 'membership',
 'benefits',
 'unlock',
 'text',
 'photos',
 'audio',
 'messages',
 'enjoy',
 'app',
 'without',
 'ads',
 'recommended',
 'users',
 'vip',
 'member',
 'unlock',
 'instant',
 'video',
 'audio',
 'calls',
 'feature',
 'video',
 'audio',
 'chat',
 'users',
 'purchasing',
 'gems',
 'gems',
 'benefits',
 'gems',
 'used',
 'video',
 'audio',
 'call',
 'gems',
 'used',
 'buy',
 'different',
 'gifts',
 'privacy'

##4. Train Validation Split

In [ ]:
# split the data into a training set and a validation set

In [ ]:
train_inputs = df_train["Clean_Description"]
validation_inputs = df_val["Clean_Description"]
train_labels = df_train[target_list]
validation_labels = df_val[target_list]

## 5. Tokenization

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_inputs)

In [ ]:
MAX_SEQUENCE_LENGTH = 600

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_inputs)

In [ ]:
train_inputs

0        enjoy chatting social video chat face chat fac...
1        running counter calorie counter app way perfec...
2        fake mask neon angel one fake mask neon angel ...
3        live wallpaper app shirokuma days  set app hom...
4        control airsoft gun smartphone tablet android ...
                               ...                        
58143    broken heart emoji theme keyboard install keyb...
58144    elementary pos turns phone tablet simple cash ...
58145    kabul calendar completely free open source pro...
58146    great watch docking station also known night t...
58147    live life rich famous mansion mansion perfect ...
Name: Clean_Description, Length: 58148, dtype: object

In [ ]:
train_sequences

[[46,
  1308,
  122,
  36,
  165,
  442,
  165,
  442,
  165,
  4253,
  9,
  5792,
  36,
  1308,
  114,
  118,
  1308,
  50,
  238,
  36,
  297,
  509,
  289,
  6230,
  79,
  10,
  722,
  504,
  46,
  12,
  442,
  165,
  204,
  442,
  165,
  165,
  114,
  122,
  43,
  112,
  12,
  350,
  384,
  265,
  36,
  165,
  232,
  83,
  22,
  963,
  525,
  172,
  2966,
  770,
  446,
  4031,
  2181,
  815,
  518,
  50,
  108,
  238,
  172,
  46,
  1,
  102,
  320,
  1386,
  91,
  4031,
  1444,
  518,
  525,
  36,
  238,
  347,
  120,
  36,
  238,
  165,
  91,
  3161,
  4630,
  4630,
  815,
  4630,
  96,
  36,
  238,
  83,
  4630,
  96,
  467,
  54,
  1655,
  179,
  546,
  28342,
  41,
  2124,
  51,
  201,
  652,
  39,
  145,
  6790,
  5252,
  2620,
  51,
  4943,
  21147,
  17098,
  7338,
  12030,
  114,
  1216,
  8823],
 [693,
  1126,
  2648,
  1126,
  1,
  67,
  200,
  322,
  1282,
  755,
  555,
  596,
  449,
  693,
  2648,
  1126,
  2,
  693,
  1,
  51,
  515,
  41,
  433,
  2191,
  322,
  555,

In [ ]:
train_padded = pad_sequences(
    train_sequences, maxlen=MAX_SEQUENCE_LENGTH, truncating="post", padding="post"
)

In [ ]:
train_padded

array([[   46,  1308,   122, ...,     0,     0,     0],
       [  693,  1126,  2648, ...,     0,     0,     0],
       [  865,  2371,   564, ...,     0,     0,     0],
       ...,
       [51660,   260,   485, ...,     0,     0,     0],
       [  174,   222, 29385, ...,     0,     0,     0],
       [   25,    93,  1542, ...,     0,     0,     0]], dtype=int32)

In [ ]:
validation_sequences = tokenizer.texts_to_sequences(validation_inputs)
validation_padded = pad_sequences(
    validation_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [ ]:
validation_padded

array([[  124,   182,    18, ...,     0,     0,     0],
       [  448,  1622,  1591, ...,     0,     0,     0],
       [   16,     8, 23133, ...,     0,     0,     0],
       ...,
       [  516,  2149,    13, ...,     0,     0,     0],
       [  762, 18455,   762, ...,     0,     0,     0],
       [  614,     1,   152, ...,     0,     0,     0]], dtype=int32)

In [ ]:
print(df_train.Clean_Description[0])
print(train_sequences[0])

enjoy chatting social video chat face chat face chat opens new doors video chatting people start chatting text audio video options anytime anywhere novel fun features waiting discover enjoy time face chat now face chat chat people social friends real time translation private secure video chat quick call easy login instant messages im beauty effect vip membership benefits unlock text photos audio messages enjoy app without ads recommended users vip member unlock instant video audio calls feature video audio chat users purchasing gems gems benefits gems used video audio call gems used buy different gifts privacy policy facechat need responsible information provide party please take caution delivering sensitive information sexual pornographic nude behaviors forbidden people rules banned
[46, 1308, 122, 36, 165, 442, 165, 442, 165, 4253, 9, 5792, 36, 1308, 114, 118, 1308, 50, 238, 36, 297, 509, 289, 6230, 79, 10, 722, 504, 46, 12, 442, 165, 204, 442, 165, 165, 114, 122, 43, 112, 12, 350, 3

In [ ]:
word_index = tokenizer.word_index
print("Number of unique words:", len(word_index))

Number of unique words: 141211


In [ ]:
word_index

{'app': 1,
 'free': 2,
 'use': 3,
 'phone': 4,
 'keyboard': 5,
 'wallpaper': 6,
 'theme': 7,
 'application': 8,
 'new': 9,
 'features': 10,
 'get': 11,
 'time': 12,
 'wallpapers': 13,
 'screen': 14,
 'also': 15,
 'best': 16,
 'one': 17,
 'like': 18,
 'make': 19,
 'download': 20,
 'android': 21,
 'easy': 22,
 'english': 23,
 'us': 24,
 'live': 25,
 'find': 26,
 'launcher': 27,
 'share': 28,
 'mobile': 29,
 'help': 30,
 'love': 31,
 'learn': 32,
 'apps': 33,
 'set': 34,
 'device': 35,
 'video': 36,
 'want': 37,
 'using': 38,
 'please': 39,
 'hd': 40,
 'need': 41,
 'support': 42,
 'friends': 43,
 'game': 44,
 'home': 45,
 'enjoy': 46,
 'simple': 47,
 'images': 48,
 'many': 49,
 'text': 50,
 'information': 51,
 'language': 52,
 'themes': 53,
 'different': 54,
 'games': 55,
 'available': 56,
 'data': 57,
 'day': 58,
 'beautiful': 59,
 'access': 60,
 'play': 61,
 'choose': 62,
 'add': 63,
 'easily': 64,
 'create': 65,
 'lock': 66,
 'way': 67,
 'you': 68,
 'words': 69,
 'save': 70,
 'every': 

In [ ]:
word_index["reason"]

2388

In [ ]:
print(validation_sequences[0])

[124, 182, 18, 3512, 31, 5, 7, 2455, 29, 4101, 449, 3512, 31, 5, 7, 633, 5501, 704, 3512, 31, 5, 7, 597, 4, 9, 146, 799, 1247, 31, 3512, 31, 5, 7, 408, 3512, 31, 5, 7, 257, 2797, 5, 59, 196, 10418, 4, 3512, 31, 5, 7, 204, 41, 279, 5, 245, 9, 29, 35, 390, 214, 155, 3512, 31, 5, 7, 4284, 1315, 2933, 3512, 31, 5, 7, 133, 803, 5150, 889, 9, 8565, 874, 43, 20, 3512, 31, 5, 7, 2, 3512, 31, 5, 7, 333, 21, 211, 171, 659, 246, 184, 111, 111, 111, 246, 184, 833, 246, 184, 833, 246, 184, 246, 184, 981, 246, 184, 10419, 740, 674, 720, 740, 674, 357, 453, 453, 361, 357, 453, 357, 453, 357, 1368, 357, 1368, 357, 1680, 721, 721, 17, 721, 2700, 10420, 26, 736, 1268, 736, 736, 1833, 3, 3512, 31, 5, 7, 5, 111, 39, 104, 5, 164, 20, 3512, 31, 5, 7, 61, 157, 107, 3512, 31, 5, 7, 98, 1055, 7, 141, 2189, 7, 5, 664, 1671, 783, 91, 637, 5, 53, 21, 2189, 5, 161, 49, 2, 294, 5, 40, 13, 617, 4, 82, 18, 516, 40, 6, 156, 40, 6, 473, 40, 6, 955, 1699, 53, 5, 256, 5, 7, 1415, 108, 40, 13, 473, 40, 13, 863, 151, 27, 1

In [ ]:
word_index["listen"]

424

## 6. Create the Embedding dictionary

In [ ]:
embedding_dict = {}
with open("/content/drive/MyDrive/glove6B/glove.6B.300d.txt", "r") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], "float32")
        embedding_dict[word] = vectors
f.close()

In [ ]:
embedding_dict

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
embedding_dim = 300

In [ ]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i < num_words:
        emb_vec = embedding_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i] = emb_vec

In [ ]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.73247999, -0.078309  ,  0.018521  , ...,  0.21988   ,
        -0.13121   ,  0.043819  ],
       [-0.30414   , -0.37029999, -0.15881   , ..., -0.37830999,
        -0.42910001,  0.0030023 ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.75204003,  0.58028001,  0.029452  , ...,  0.022623  ,
        -1.11360002, -0.17218   ]])

In [ ]:
word_index["reason"]

2388

In [ ]:
embedding_dict.get("reason")

array([ 1.3197e-01, -1.2591e-01,  4.3864e-02,  3.6321e-02,  9.6646e-02,
       -1.3829e-01,  3.8637e-01,  7.6962e-02, -1.1306e-01, -1.6083e+00,
        2.0062e-02, -5.2665e-02, -1.6597e-01,  1.2171e-01,  2.8945e-01,
       -1.7289e-01,  5.3035e-02, -2.7842e-01,  8.2376e-02, -1.1980e-02,
        3.7228e-02,  2.1867e-01,  1.5267e-01, -8.4361e-02, -3.1292e-01,
       -3.2093e-02,  2.0281e-01, -3.5910e-01,  1.6873e-02, -2.2996e-01,
        2.6044e-02,  3.5910e-01, -3.2431e-01, -5.4194e-01, -9.7742e-01,
        3.9198e-02, -1.7794e-01,  7.4200e-02, -4.1251e-02, -7.8917e-02,
        2.0646e-01, -6.6538e-02,  6.7401e-02,  1.4965e-01,  5.9107e-02,
       -3.7585e-02, -3.4672e-02,  5.5291e-02, -8.5636e-02,  9.1743e-02,
        4.9125e-01,  7.5606e-03, -3.0860e-01,  5.8902e-04, -8.6975e-02,
        3.9904e-01, -1.2695e-01,  2.2471e-01,  2.3658e-01,  3.0489e-01,
       -6.7363e-02,  3.5839e-01,  4.9703e-01,  4.1895e-01, -3.8494e-01,
       -2.6257e-01,  1.6049e-01, -1.0992e-01,  2.7477e-02,  1.49

In [ ]:
(embedding_matrix[2363] == embedding_dict.get("reason")).all()

False

In [ ]:
print(train_padded.shape)
print(train_labels.shape)

(58148, 600)
(58148, 9)


In [ ]:
print(validation_padded.shape)
print(validation_labels.shape)

(14536, 600)
(14536, 9)


## 7. Build CNN Model

In [ ]:
num_permissions = 9 #11
drop = 0.2

batch_size = 32
max_train_epochs = 300
validation_split = 6

early_stopping_patience = 16
early_stopping_delta = 0.02  # 2%

max_description_embeddings = 600
embedding_dim = 300  # +1 for flag

#downloaded_embedding_file = data_folder + "/word_embeddings/glove.6B.300d.txt"


conv_filters_num = 1024
conv_filters_sizes = [1, 2, 3]
dense_layers = [5000, 2500]
dropout = 0.2

heatmap_threshold = 0.49

In [ ]:
# Function for class weights

import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import MultiLabelBinarizer


def generate_class_weights(class_series, multi_class=True, one_hot_encoded=False):
  """
  Method to generate class weights given a set of multi-class or multi-label labels, both one-hot-encoded or not.
  Some examples of different formats of class_series and their outputs are:
    - generate_class_weights(['mango', 'lemon', 'banana', 'mango'], multi_class=True, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 1.3333333333333333, 'mango': 0.6666666666666666}
    - generate_class_weights([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0]], multi_class=True, one_hot_encoded=True)
    {0: 0.6666666666666666, 1: 1.3333333333333333, 2: 1.3333333333333333}
    - generate_class_weights([['mango', 'lemon'], ['mango'], ['lemon', 'banana'], ['lemon']], multi_class=False, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 0.4444444444444444, 'mango': 0.6666666666666666}
    - generate_class_weights([[0, 1, 1], [0, 0, 1], [1, 1, 0], [0, 1, 0]], multi_class=False, one_hot_encoded=True)
    {0: 1.3333333333333333, 1: 0.4444444444444444, 2: 0.6666666666666666}
  The output is a dictionary in the format { class_label: class_weight }. In case the input is one hot encoded, the class_label would be index
  of appareance of the label when the dataset was processed. 
  In multi_class this is np.unique(class_series) and in multi-label np.unique(np.concatenate(class_series)).
  Author: Angel Igareta (angel@igareta.com)
  """
  if multi_class:
    # If class is one hot encoded, transform to categorical labels to use compute_class_weight   
    if one_hot_encoded:
      class_series = np.argmax(class_series, axis=1)
  
    # Compute class weights with sklearn method
    class_labels = np.unique(class_series)
    class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=class_series)

    print(class_series)
    print(class_labels)

    return dict(zip(class_labels, class_weights))
  else:
    # It is neccessary that the multi-label values are one-hot encoded
    mlb = None
    if not one_hot_encoded:
      mlb = MultiLabelBinarizer()
      class_series = mlb.fit_transform(class_series)

    n_samples = len(class_series)
    n_classes = len(class_series[0])
    print(n_samples)
    print(n_classes)

    # Count each class frequency
    class_count = [0] * n_classes
    for classes in class_series:
        for index in range(n_classes):
            if classes[index] != 0:
                class_count[index] += 1
    
    # Compute class weights using balanced method
    class_weights = [n_samples / (n_classes * freq) if freq > 0 else 1 for freq in class_count]
    class_labels = range(len(class_weights)) if mlb is None else mlb.classes_
    return dict(zip(class_labels, class_weights))
    #return class_weights

In [ ]:
class_series = np.array(train_labels)
class_wt = generate_class_weights(class_series, multi_class=False, one_hot_encoded=True)
print(class_wt)
#class_wt = torch.tensor(class_wt)
#print(class_wt)

58148
9
{0: 0.7022705314009662, 1: 0.7092862980446689, 2: 1.196017935743963, 3: 1.0206775495875022, 4: 0.22669785575048734, 5: 0.7239902385576971, 6: 23.157307845479888, 7: 38.23011176857331, 8: 6.266623558573122}


In [ ]:
def model_multiconv_1d(num_permissions):
    #embedding_dim = embedding_dim
    sequence_length = max_description_embeddings

    input_layer = Input(shape=(None,))

    conv_layers = []
    for filter_size in conv_filters_sizes:
        conv_layer_i = Embedding(num_words,
                                 output_dim=embedding_dim,
                                 input_length=sequence_length,
                                 weights=[embedding_matrix],
                                 trainable=False)(input_layer)
        conv_layer_i = Conv1D(filters=conv_filters_num,
                              kernel_size=filter_size,
                              padding='same',
                              activation='relu')(conv_layer_i)
        conv_layer_i = GlobalMaxPooling1D()(conv_layer_i)

        conv_layers.append(conv_layer_i)

    if len(conv_layers) == 1:
        previous_layer = conv_layers[0]
    else:
        concatenated_layer = concatenate(conv_layers, axis=-1)
        previous_layer = concatenated_layer

    for n_neurons in dense_layers:
        previous_layer = Dense(n_neurons, activation='relu')(previous_layer)
        previous_layer = Dropout(dropout)(previous_layer)

    output_layer = Dense(num_permissions, activation='sigmoid')(previous_layer)

    return keras.Model(inputs=input_layer, outputs=output_layer)

In [ ]:
model = model_multiconv_1d(num_permissions)

In [ ]:
model.compile(loss="binary_crossentropy", optimizer=Adam(0.0001), metrics=['accuracy'])
              
#metrics=[metrics.fb_micro, metrics.fb_macro, metrics.precision, metrics.recall])
train_metric = 'val_fb_macro'

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    42363600    ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    42363600    ['input_1[0][0]']                
                                                                                                  
 embedding_2 (Embedding)        (None, None, 300)    42363600    ['input_1[0][0]']                
                                                                                              

In [ ]:
checkpoint = ModelCheckpoint('weights_cnn_sentece.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

model.fit(train_padded, train_labels, 
          batch_size=batch_size, epochs=10, class_weight=class_wt,
          verbose=1, callbacks=[checkpoint], validation_data=(validation_padded, validation_labels))

Epoch 1/10
1818/1818 [==============================] - 363s 193ms/step - loss: 0.1740 - accuracy: 0.2628 - val_loss: 0.2538 - val_accuracy: 0.2534
Epoch 2/10
1818/1818 [==============================] - 339s 187ms/step - loss: 0.1426 - accuracy: 0.2849 - val_loss: 0.2384 - val_accuracy: 0.2937
Epoch 3/10
1818/1818 [==============================] - 339s 186ms/step - loss: 0.1208 - accuracy: 0.3007 - val_loss: 0.2281 - val_accuracy: 0.2735
Epoch 4/10
1818/1818 [==============================] - 338s 186ms/step - loss: 0.0965 - accuracy: 0.3050 - val_loss: 0.2422 - val_accuracy: 0.2920
Epoch 5/10
1818/1818 [==============================] - 338s 186ms/step - loss: 0.0815 - accuracy: 0.3095 - val_loss: 0.2586 - val_accuracy: 0.3041
Epoch 6/10
1818/1818 [==============================] - 338s 186ms/step - loss: 0.0686 - accuracy: 0.3221 - val_loss: 0.2486 - val_accuracy: 0.3247
Epoch 7/10
1818/1818 [==============================] - 338s 186ms/step - loss: 0.0574 - accuracy: 0.3398 - val_

## 8. Performance on Test Set

In [ ]:
test_labels = df_test[target_list]
test_input = df_test["Clean_Description"]

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test_input)
test_padded = pad_sequences(
    test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [ ]:
print(df_test.Clean_Description[0])
print(test_sequences[0])

poetry lover find best poetry poets urdu problem solved app app contains time best urdu poetry famous urdu poets offline urdu poetry collection best urdu poetry famous urdu poets like mohsin naqvi mirza ghalib saghir siddiqui others also contains romantic urdu poetry sad urdu poetry urdu ghazals offline urdu shayari urdu shayari app contains best ever poetry famous poets times app contains best variety urdu poetry categories romantic poetry sad poetry attitude poetry ghazals inspirational poetry main features offline poetry collection gallery view poetry collection zooming feature save share poetry images app contains offline poetry urdu ghazals urdu poetry poetry famous poets urdu shayari much more
[2887, 1849, 26, 16, 2887, 10999, 365, 678, 2532, 1, 1, 212, 12, 16, 365, 2887, 1014, 365, 10999, 149, 365, 2887, 137, 16, 365, 2887, 1014, 365, 10999, 18, 35036, 47369, 19907, 18506, 50841, 359, 15, 212, 544, 365, 2887, 1559, 365, 2887, 365, 9403, 149, 365, 625, 365, 625, 1, 212, 16, 515, 

In [ ]:
print(test_labels)

      Camera  Location  Microphone  Contacts  Storage  Phone  SMS  Call_Log  \
0          0         0           0         0        1      0    0         0   
1          0         0           0         0        1      0    0         0   
2          0         0           0         0        1      0    0         0   
3          0         0           0         0        0      0    0         0   
4          1         0           0         0        0      1    0         0   
...      ...       ...         ...       ...      ...    ...  ...       ...   
4619       0         1           0         0        0      0    0         0   
4620       0         0           0         0        0      0    0         0   
4621       0         0           0         0        1      0    0         0   
4622       0         1           1         0        1      0    0         0   
4623       0         0           0         0        0      0    0         0   

      Calendar  
0            0  
1            0  


In [ ]:
print(test_padded.shape)
print(test_labels.shape)

(4624, 600)
(4624, 9)


In [ ]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`

predictions = model.predict(test_padded)
print("predictions shape:", predictions.shape)

predictions shape: (4624, 9)


In [ ]:
predictions[0:10]

array([[2.58654077e-03, 7.97963329e-03, 1.77200334e-08, 7.12214430e-08,
        3.08367968e-01, 7.95255255e-05, 2.68688995e-11, 5.61851320e-14,
        5.74025982e-09],
       [3.73296348e-06, 7.13312111e-05, 5.21453444e-07, 4.43870164e-07,
        1.64377084e-03, 2.81526918e-05, 5.15771914e-09, 4.76496577e-12,
        6.80708183e-07],
       [9.74794507e-01, 1.15353778e-01, 9.57949162e-02, 4.98861969e-02,
        9.85489190e-01, 4.45890397e-01, 5.86755268e-05, 1.22479987e-05,
        3.85949714e-03],
       [2.26341130e-04, 2.49230210e-02, 1.90126195e-06, 1.06700294e-01,
        7.22305536e-01, 2.71699786e-01, 7.71374471e-06, 2.29562806e-06,
        4.11510919e-05],
       [1.87917217e-03, 5.67462048e-05, 1.57249335e-04, 4.64897417e-03,
        6.05917478e-04, 2.43818294e-02, 2.66471034e-04, 3.39141093e-06,
        2.24964591e-04],
       [2.18619600e-01, 1.78040820e-03, 2.28648707e-02, 1.40656868e-03,
        9.81777251e-01, 2.27748770e-02, 1.33883384e-06, 9.20022956e-08,
        1.1

In [ ]:
true_labels = test_labels.to_numpy()
true_labels[0:10]

array([[0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0]])

In [ ]:
np.save("predictions.npy", predictions)
loaded_predictions = np.load("predictions.npy")
print(loaded_predictions)

[[2.5865408e-03 7.9796333e-03 1.7720033e-08 ... 2.6868899e-11
  5.6185132e-14 5.7402598e-09]
 [3.7329635e-06 7.1331211e-05 5.2145344e-07 ... 5.1577191e-09
  4.7649658e-12 6.8070818e-07]
 [9.7479451e-01 1.1535378e-01 9.5794916e-02 ... 5.8675527e-05
  1.2247999e-05 3.8594971e-03]
 ...
 [9.8308748e-01 5.7583820e-02 1.0786277e-02 ... 1.6441523e-05
  1.6933303e-06 1.5318161e-03]
 [3.3212514e-04 9.9983203e-01 9.9993956e-01 ... 7.7595060e-17
  3.1444198e-16 3.2073799e-06]
 [1.8154529e-04 9.7847969e-06 2.2670772e-06 ... 2.7730421e-08
  1.1940759e-10 3.0523313e-07]]


In [ ]:
np.save("true_labels.npy", true_labels)
loaded_true_labels = np.load("true_labels.npy")
print(loaded_true_labels)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## 9. Threshold Calculation

In [ ]:
# Import module for data manipulation
import pandas as pd
# Import module for linear algebra
import numpy as np
# Import module for data simulation
from sklearn.datasets import make_classification     # Create a synthetic dataframe
from sklearn.linear_model import LogisticRegression  # Classification model
from sklearn.model_selection import train_test_split # Split the dataframe
from sklearn.metrics import roc_curve                # Calculate the ROC curve
from sklearn.metrics import precision_recall_curve   # Calculate the Precision-Recall curve
from sklearn.metrics import f1_score                 # Calculate the F-score
# Import module for data visualization
from plotnine import *
import plotnine

In [ ]:
thresh_f = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
thresh_roc = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [ ]:
n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("predictions.npy")
  true_labels = np.load("true_labels.npy")

  # Array for finding the optimal threshold
  thresholds = np.arange(0.0, 1.0, 0.0001)
  fscore = np.zeros(shape=(len(thresholds)))
  print('Length of sequence: {}'.format(len(thresholds)))

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Fit the model
  for index, elem in enumerate(thresholds):
    # Corrected probabilities
    y_pred_prob = (pred > elem).astype('int')
    # Calculate the f-score
    fscore[index] = f1_score(labels, y_pred_prob)

  # Find the optimal threshold
  index = np.argmax(fscore)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  fscoreOpt = round(fscore[index], ndigits = 4)
  thresh_f[i] = thresholdOpt
  print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))

print("-------------------------------------")
print("optimal threshold tuning for f-score")
print(thresh_f)

permission  0
Length of sequence: 10000
Best Threshold: 0.4029 with F-Score: 0.6642
permission  1
Length of sequence: 10000
Best Threshold: 0.0877 with F-Score: 0.5637
permission  2
Length of sequence: 10000
Best Threshold: 0.0514 with F-Score: 0.5753
permission  3
Length of sequence: 10000
Best Threshold: 0.0126 with F-Score: 0.489
permission  4
Length of sequence: 10000
Best Threshold: 0.0982 with F-Score: 0.7954
permission  5
Length of sequence: 10000
Best Threshold: 0.0844 with F-Score: 0.5159
permission  6
Length of sequence: 10000
Best Threshold: 0.0362 with F-Score: 0.0952
permission  7
Length of sequence: 10000
Best Threshold: 0.231 with F-Score: 0.2857
permission  8
Length of sequence: 10000
Best Threshold: 0.096 with F-Score: 0.3485
-------------------------------------
optimal threshold tuning for f-score
[0.4029, 0.0877, 0.0514, 0.0126, 0.0982, 0.0844, 0.0362, 0.231, 0.096]


In [ ]:
##for roc curve with g-mean

n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("predictions.npy")
  true_labels = np.load("true_labels.npy")

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Create the ROC curve
  fpr, tpr, thresholds = roc_curve(labels, pred)

  df_fpr_tpr = pd.DataFrame({'FPR':fpr, 'TPR':tpr, 'Threshold':thresholds})

  # Calculate the G-mean
  gmean = np.sqrt(tpr * (1 - fpr))

  # Find the optimal threshold
  index = np.argmax(gmean)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  gmeanOpt = round(gmean[index], ndigits = 4)
  fprOpt = round(fpr[index], ndigits = 4)
  tprOpt = round(tpr[index], ndigits = 4)

  thresh_roc[i] = thresholdOpt
  print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
  print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))

print("-------------------------------------")
print("ROC curve with G-mean threshold tuning")
print(thresh_roc)

permission  0
Best Threshold: 0.06120000034570694 with G-Mean: 0.8421
FPR: 0.172, TPR: 0.8564
permission  1
Best Threshold: 0.00570000009611249 with G-Mean: 0.7636
FPR: 0.2559, TPR: 0.7835
permission  2
Best Threshold: 0.003100000089034438 with G-Mean: 0.8229
FPR: 0.1516, TPR: 0.7982
permission  3
Best Threshold: 0.0007999999797903001 with G-Mean: 0.7787
FPR: 0.2124, TPR: 0.77
permission  4
Best Threshold: 0.40290001034736633 with G-Mean: 0.78
FPR: 0.1958, TPR: 0.7565
permission  5
Best Threshold: 0.00930000003427267 with G-Mean: 0.76
FPR: 0.2558, TPR: 0.7761
permission  6
Best Threshold: 0.0 with G-Mean: 0.8967
FPR: 0.196, TPR: 1.0
permission  7
Best Threshold: 0.0 with G-Mean: 0.9227
FPR: 0.1485, TPR: 1.0
permission  8
Best Threshold: 0.0003000000142492354 with G-Mean: 0.8551
FPR: 0.1773, TPR: 0.8889
-------------------------------------
ROC curve with G-mean threshold tuning
[0.0612, 0.0057, 0.0031, 0.0008, 0.4029, 0.0093, 0.0, 0.0, 0.0003]


## 10. Performance Score

In [ ]:
#Fscore micro for different thresholds-

In [ ]:
#predictions = np.load("predictions.npy")
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [ ]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import f1_score

def f_at_1(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_f[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_f[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_f[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_f[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_f[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_f[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_f[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_f[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_f[8]

   
    acc[0] = f1_score(labels[:, 0], preds_th[:, 0])
    acc[1] = f1_score(labels[:, 1], preds_th[:, 1])
    acc[2] = f1_score(labels[:, 2], preds_th[:, 2])
    acc[3] = f1_score(labels[:, 3], preds_th[:, 3])
    acc[4] = f1_score(labels[:, 4], preds_th[:, 4])
    acc[5] = f1_score(labels[:, 5], preds_th[:, 5])
    acc[6] = f1_score(labels[:, 6], preds_th[:, 6])
    acc[7] = f1_score(labels[:, 7], preds_th[:, 7])
    acc[8] = f1_score(labels[:, 8], preds_th[:, 8])

   #f1_score_micro = metrics.f1_score(targets, outputs, average='micro')

    #print(acc)
    return acc

In [ ]:
eval_accuracy = f_at_1(predictions, true_labels)

np.save("F1_CV1_N72k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.
print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")
avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 score: {0:.4f}".format(avg_score))

  Camera    : 0.6642
  Location  : 0.5637
  Microphone: 0.5753
  Contacts  : 0.4890
  Storage   : 0.7954
  Phone     : 0.5159
  SMS       : 0.0952
  Call_Log  : 0.2857
  Calendar  : 0.3485

  Average F1 score: 0.4814


In [ ]:
#Fscore micro for different thresholds-

In [ ]:
#predictions = np.load("predictions.npy")
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [ ]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import f1_score

def f1micro_accuracy(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_f[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_f[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_f[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_f[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_f[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_f[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_f[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_f[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_f[8]

    acc[0] = f1_score(labels[:, 0], preds_th[:, 0], average='micro')
    acc[1] = f1_score(labels[:, 1], preds_th[:, 1], average='micro')
    acc[2] = f1_score(labels[:, 2], preds_th[:, 2], average='micro')
    acc[3] = f1_score(labels[:, 3], preds_th[:, 3], average='micro')
    acc[4] = f1_score(labels[:, 4], preds_th[:, 4], average='micro')
    acc[5] = f1_score(labels[:, 5], preds_th[:, 5], average='micro')
    acc[6] = f1_score(labels[:, 6], preds_th[:, 6], average='micro')
    acc[7] = f1_score(labels[:, 7], preds_th[:, 7], average='micro')
    acc[8] = f1_score(labels[:, 8], preds_th[:, 8], average='micro')
    
   #f1_score_micro = metrics.f1_score(targets, outputs, average='micro')

    #print(acc)
    return acc

In [ ]:
eval_accuracy = f1micro_accuracy(predictions, true_labels)

np.save("F1Mic_CV1_N72k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 (micro) score: {0:.4f}".format(avg_score))

  Camera    : 0.8828
  Location  : 0.8571
  Microphone: 0.9109
  Contacts  : 0.8748
  Storage   : 0.7753
  Phone     : 0.8523
  SMS       : 0.9959
  Call_Log  : 0.9989
  Calendar  : 0.9814

  Average F1 (micro) score: 0.9033


In [ ]:
#roc-auc score for different thresholds-

In [ ]:
import numpy as np
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [ ]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import roc_auc_score

def roc_auc(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_roc[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_roc[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_roc[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_roc[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_roc[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_roc[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_roc[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_roc[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_roc[8]


    acc[0] = roc_auc_score(labels[:, 0], preds_th[:, 0])
    acc[1] = roc_auc_score(labels[:, 1], preds_th[:, 1])
    acc[2] = roc_auc_score(labels[:, 2], preds_th[:, 2])
    acc[3] = roc_auc_score(labels[:, 3], preds_th[:, 3])
    acc[4] = roc_auc_score(labels[:, 4], preds_th[:, 4])
    acc[5] = roc_auc_score(labels[:, 5], preds_th[:, 5])
    acc[6] = roc_auc_score(labels[:, 6], preds_th[:, 6])
    acc[7] = roc_auc_score(labels[:, 7], preds_th[:, 7])
    acc[8] = roc_auc_score(labels[:, 8], preds_th[:, 8])
   

    #print(acc)
    return acc

In [ ]:
#roc-auc score

eval_accuracy = roc_auc(predictions, true_labels)

np.save("ROC_CV1_N72k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average ROC_AUC score: {0:.4f}".format(avg_score))

  Camera    : 0.8415
  Location  : 0.7638
  Microphone: 0.8221
  Contacts  : 0.7772
  Storage   : 0.7803
  Phone     : 0.7596
  SMS       : 0.5000
  Call_Log  : 0.5000
  Calendar  : 0.8531

  Average ROC_AUC score: 0.7331


# ACNET performance

#### a) Data Preparation

In [ ]:
## drive path
acnet_path = "/content/drive/MyDrive/MetadataCSV/acnet_dataset_preprocess.csv"

In [ ]:
df_acnet = pd.read_csv(acnet_path) 
print(df_acnet.shape)

(1417, 14)


In [ ]:
df_acnet = df_acnet.dropna(subset=['Clean_Description'])

In [ ]:
df_acnet.head()

,app_id,description,Storage,Contacts,Location,Camera,Microphone,SMS,Call_Log,Phone,Calendar,Settings,Tasks,Clean_Description
0,0,ROOT is REQUIRED for automatic synchronization...,1,0,1,0,0,0,0,0,0,0,0,root is required for automatic synchronization...
1,1,This app delivers short scriptures containing ...,0,0,0,0,0,1,0,0,0,0,0,this app delivers short scriptures containing ...
2,2,This game is surprisingly simple and very addi...,0,0,0,0,0,0,0,0,0,0,0,this game is surprisingly simple and very addi...
3,3,It is an online RPG game based on LBS location...,0,0,1,0,0,0,0,0,0,0,0,it is an online rpg game based on lbs location...
4,4,Christmas is in the air. Get yourself in the h...,0,1,0,0,0,0,0,0,0,1,1,christmas is in the air. get yourself in the h...


In [ ]:
df_acnet["Clean_Description"] = df_acnet["Clean_Description"].map(remove_stopwords)

In [ ]:
df_acnet["Clean_Description"] = df_acnet["Clean_Description"].map(lambda x: remove_punct(x))

In [ ]:
acnet_labels = df_acnet[target_list]
acnet_input = df_acnet["Clean_Description"]

In [ ]:
acnet_sequences = tokenizer.texts_to_sequences(acnet_input)
acnet_padded = pad_sequences(
    acnet_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [ ]:
print(acnet_padded.shape)
print(acnet_labels.shape)

(1414, 600)
(1414, 9)


#### b) Get Predictions

In [ ]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`

predictions = model.predict(acnet_padded)
print("predictions shape:", predictions.shape)

predictions shape: (1414, 9)


In [ ]:
acnet_labels = acnet_labels.to_numpy()

In [ ]:
np.save("acnet_predictions.npy", predictions)
loaded_predictions = np.load("acnet_predictions.npy")

np.save("acnet_labels.npy", acnet_labels)
loaded_true_labels = np.load("acnet_labels.npy")

#### c) Threshold Calculation

In [ ]:
thresh_f = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
thresh_roc = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [ ]:
n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("acnet_predictions.npy")
  true_labels = np.load("acnet_labels.npy")

  # Array for finding the optimal threshold
  thresholds = np.arange(0.0, 1.0, 0.0001)
  fscore = np.zeros(shape=(len(thresholds)))
  print('Length of sequence: {}'.format(len(thresholds)))

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Fit the model
  for index, elem in enumerate(thresholds):
    # Corrected probabilities
    y_pred_prob = (pred > elem).astype('int')
    # Calculate the f-score
    fscore[index] = f1_score(labels, y_pred_prob)

  # Find the optimal threshold
  index = np.argmax(fscore)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  fscoreOpt = round(fscore[index], ndigits = 4)
  thresh_f[i] = thresholdOpt
  print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))

print("-------------------------------------")
print("optimal threshold tuning for f-score")
print(thresh_f)

permission  0
Length of sequence: 10000
Best Threshold: 0.6837 with F-Score: 0.4421
permission  1
Length of sequence: 10000
Best Threshold: 0.1645 with F-Score: 0.5535
permission  2
Length of sequence: 10000
Best Threshold: 0.6347 with F-Score: 0.5627
permission  3
Length of sequence: 10000
Best Threshold: 0.0071 with F-Score: 0.6101
permission  4
Length of sequence: 10000
Best Threshold: 0.0001 with F-Score: 0.5834
permission  5
Length of sequence: 10000
Best Threshold: 0.7136 with F-Score: 0.4095
permission  6
Length of sequence: 10000
Best Threshold: 0.0001 with F-Score: 0.5703
permission  7
Length of sequence: 10000
Best Threshold: 0.0063 with F-Score: 0.3375
permission  8
Length of sequence: 10000
Best Threshold: 0.0463 with F-Score: 0.5699
-------------------------------------
optimal threshold tuning for f-score
[0.6837, 0.1645, 0.6347, 0.0071, 0.0001, 0.7136, 0.0001, 0.0063, 0.0463]


In [ ]:
##for roc curve with g-mean

n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("acnet_predictions.npy")
  true_labels = np.load("acnet_labels.npy")

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Create the ROC curve
  fpr, tpr, thresholds = roc_curve(labels, pred)

  df_fpr_tpr = pd.DataFrame({'FPR':fpr, 'TPR':tpr, 'Threshold':thresholds})

  # Calculate the G-mean
  gmean = np.sqrt(tpr * (1 - fpr))

  # Find the optimal threshold
  index = np.argmax(gmean)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  gmeanOpt = round(gmean[index], ndigits = 4)
  fprOpt = round(fpr[index], ndigits = 4)
  tprOpt = round(tpr[index], ndigits = 4)

  thresh_roc[i] = thresholdOpt
  print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
  print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))

print("-------------------------------------")
print("ROC curve with G-mean threshold tuning")
print(thresh_roc)

permission  0
Best Threshold: 0.20059999823570251 with G-Mean: 0.6885
FPR: 0.2616, TPR: 0.6419
permission  1
Best Threshold: 0.056299999356269836 with G-Mean: 0.7245
FPR: 0.2216, TPR: 0.6743
permission  2
Best Threshold: 0.005400000140070915 with G-Mean: 0.7955
FPR: 0.2259, TPR: 0.8176
permission  3
Best Threshold: 0.005799999926239252 with G-Mean: 0.7144
FPR: 0.2155, TPR: 0.6506
permission  4
Best Threshold: 0.49480000138282776 with G-Mean: 0.5576
FPR: 0.4194, TPR: 0.5355
permission  5
Best Threshold: 0.0729999989271164 with G-Mean: 0.7245
FPR: 0.2877, TPR: 0.7368
permission  6
Best Threshold: 0.0 with G-Mean: 0.7889
FPR: 0.2238, TPR: 0.8019
permission  7
Best Threshold: 0.0 with G-Mean: 0.7598
FPR: 0.2511, TPR: 0.7708
permission  8
Best Threshold: 0.005799999926239252 with G-Mean: 0.8399
FPR: 0.116, TPR: 0.7981
-------------------------------------
ROC curve with G-mean threshold tuning
[0.2006, 0.0563, 0.0054, 0.0058, 0.4948, 0.073, 0.0, 0.0, 0.0058]


#### d) Accuracy Score

In [ ]:
#Fscore micro for different thresholds-

In [ ]:
#predictions = np.load("predictions.npy")
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [ ]:
eval_accuracy = f_at_1(predictions, true_labels)

np.save("ACNET_F1_CV1_N72k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.
print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")
avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 score: {0:.4f}".format(avg_score))

  Camera    : 0.4421
  Location  : 0.5535
  Microphone: 0.5627
  Contacts  : 0.6101
  Storage   : 0.5834
  Phone     : 0.4095
  SMS       : 0.5703
  Call_Log  : 0.3375
  Calendar  : 0.5699

  Average F1 score: 0.5155


In [ ]:
#Fscore micro for different thresholds-

In [ ]:
#predictions = np.load("predictions.npy")
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [ ]:
eval_accuracy = f1micro_accuracy(predictions, true_labels)

np.save("ACNET_F1Mic_CV1_N72k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 (micro) score: {0:.4f}".format(avg_score))

  Camera    : 0.8091
  Location  : 0.7992
  Microphone: 0.9187
  Contacts  : 0.7496
  Storage   : 0.4173
  Phone     : 0.9123
  SMS       : 0.8402
  Call_Log  : 0.9250
  Calendar  : 0.9413

  Average F1 (micro) score: 0.8125


In [ ]:
#roc-auc score for different thresholds-

In [ ]:
import numpy as np
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [ ]:
#roc-auc score

eval_accuracy = roc_auc(predictions, true_labels)

np.save("ACNET_ROC_CV1_N72k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average ROC_AUC score: {0:.4f}".format(avg_score))

  Camera    : 0.6902
  Location  : 0.7247
  Microphone: 0.7958
  Contacts  : 0.7164
  Storage   : 0.5572
  Phone     : 0.7202
  SMS       : 0.5000
  Call_Log  : 0.5000
  Calendar  : 0.8403

  Average ROC_AUC score: 0.6716
